In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="eVeIuFmVXuAoqDnlp9kf")
project = rf.workspace("traffic-analysis-zrxob").project("final-year-plxuh")
version = project.version(4)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.1/83.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Final-year-4 in yolov8:: 100%|██████████| 39424/39424 [00:05<00:00, 7830.38it/s] 


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.6/922.6 kB 24.5 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
from ultralytics import YOLO
from ultralytics.nn.modules import Conv, C2f, DWConv
import numpy as np

class PrecisionOptimizedDySIGN(torch.nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv_path = nn.Sequential(
            Conv(channels, channels, k=3, act="SiLU"),
            Conv(channels, channels, k=3, act="SiLU")
        )
        self.attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            Conv(channels, max(16, channels//8)),  # Removed redundant act
            Conv(max(16, channels//8), channels, act="Sigmoid")
        )
        
    def forward(self, x):
        return x + self.conv_path(x) * self.attention(x)
        
class PrecisionNeck(torch.nn.Module):
    """High-precision neck architecture"""
    def __init__(self, channels=[256, 512, 1024]):
        super().__init__()
        # Top-down path
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.td_blocks = nn.ModuleList([
            C2f(channels[i]+channels[i-1], channels[i], n=1)
            for i in range(1, len(channels))
        ])
        
        # Bottom-up path
        self.downsample = nn.MaxPool2d(2, stride=2)
        self.bu_blocks = nn.ModuleList([
            C2f(channels[i]+channels[i+1], channels[i], n=1)
            for i in range(len(channels)-1)
        ])
        
        # Precision boosters
        self.prec_blocks = nn.ModuleList([
            PrecisionOptimizedDySIGN(ch) for ch in channels
        ])

    def forward(self, features):
        c3, c4, c5 = features
        
        # Top-down
        p5 = self.prec_blocks[2](c5)
        p4 = self.td_blocks[0](torch.cat([c4, self.upsample(p5)], 1))
        p4 = self.prec_blocks[1](p4)
        p3 = self.td_blocks[1](torch.cat([c3, self.upsample(p4)], 1))
        p3 = self.prec_blocks[0](p3)
        
        # Bottom-up
        n3 = p3
        n4 = self.bu_blocks[0](torch.cat([p4, self.downsample(n3)], 1))
        n5 = self.bu_blocks[1](torch.cat([p5, self.downsample(n4)], 1))
        
        return [n3, n4, n5]

class PrecisionYOLO(YOLO):
    def __init__(self, model_path='yolov8s.pt'):
        super().__init__(model_path)
        self.enhance_model()
    
    def enhance_model(self):
        # Get current head structure
        head = self.model.model[-1]
        
        # Get input channels from first Conv in detection head
        for m in head.modules():
            if isinstance(m, Conv):
                in_channels = m.conv.in_channels
                break
        
        # Insert precision block before existing head
        head.cv2 = nn.Sequential(
            PrecisionOptimizedDySIGN(in_channels),
            head.cv2
        )

# Strategic Training Configuration
model = PrecisionYOLO('yolov8s.pt')

model.train(
    data='/kaggle/working/Final-year-4/data.yaml',
    epochs=200,
    batch=20 if torch.cuda.is_available() else 12,
    imgsz=640,
    lr0=0.002,  # Higher initial LR with careful decay
    lrf=0.01,    # Cosine-like decay
    optimizer='AdamW',
    weight_decay=0.001,
    momentum=0.9,
    augment=True,
    mixup=0.15,
    # cutmix=0.1,   # New augmentation
    close_mosaic=20,
    pretrained=True,
    amp=True,
    warmup_epochs=5,  # Critical for stability
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,
    box=7.5,  # Tuned loss weights
    cls=1.5,
    dfl=1.5,
    # fl_gamma=1.5,  # Focal loss
    label_smoothing=0.15,
    patience=35,
    device=[0] if torch.cuda.is_available() else None
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 21.5M/21.5M [00:00<00:00, 259MB/s]


WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.86 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/Final-year-4/data.yaml, epochs=200, time=None, patience=35, batch=20, imgsz=640, save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=20, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fram

100%|██████████| 755k/755k [00:00<00:00, 26.2MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 95/95 [00:55<00:00,  1.72it/s]


                   all       3784       4729      0.774      0.743      0.775      0.547
         License_Plate       3293       3382      0.956      0.981      0.992      0.619
                   bus         57         69      0.723      0.609      0.647      0.517
                   car        200        644       0.71      0.877      0.855      0.626
             motorbike         34         52      0.812      0.415      0.546      0.274
                person        118        498        0.9      0.992      0.987       0.76
                 truck         65         84      0.542      0.583      0.625      0.486


invalid value encountered in less
invalid value encountered in less


Speed: 0.2ms preprocess, 11.5ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78831a5fd660>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     